# **Case 1 – Migration & Default Risk**
> **Credit, Complexity and Systemic Risk**

In [ ]:
# Imports –––––
import numpy as np
import pandas as pd
from scipy.stats import norm


## **Data import**

In [2]:
# Migration & Default Assumptions –––––

# Ratings
vROWS = ["AAA","AA","A","BBB","BB","B","CCC"]
vCOLS = ["AAA","AA","A","BBB","BB","B","CCC","D"]

# Migration probabilities (in %)
mPROB = [
    [91.115, 8.179, 0.607, 0.072, 0.024, 0.003, 0.000, 0.000],  # AAA
    [0.844, 89.626, 8.954, 0.437, 0.064, 0.036, 0.018, 0.021],  # AA
    [0.055, 2.595, 91.138, 5.509, 0.499, 0.107, 0.045, 0.052],  # A
    [0.031, 0.147, 4.289, 90.584, 3.898, 0.708, 0.175, 0.168],  # BBB
    [0.007, 0.044, 0.446, 6.741, 83.274, 7.667, 0.895, 0.926],  # BB
    [0.008, 0.031, 0.150, 0.490, 5.373, 82.531, 7.894, 3.523],  # B
    [0.000, 0.015, 0.023, 0.091, 0.388, 7.630, 83.035, 8.818],  # CCC
]

# Create DataFrame
dfTRANSITIONS = pd.DataFrame(mPROB, index=vROWS, columns=vCOLS)

# Reshape DataFrame to long format
# dfTRANSITIONS = (
#     dfTRANSITIONS.reset_index(names="FROM")
#       .melt(id_vars="FROM", var_name="TO", value_name="PROB")
# )

# Convert probabilities from % to decimals
# dfTRANSITIONS["PROB"] = dfTRANSITIONS["PROB"] / 100

# Convert probabilities from % to decimals
dfTRANSITIONS = dfTRANSITIONS / 100

# Drop unused variables
del vROWS, vCOLS, mPROB


In [3]:
# Bond Valuations –––––

# Create DataFrame
dfBONDVALUES = pd.DataFrame(
    {
        "RATING": ["AAA","AA","A","BBB","BB","B","CCC","D"],
        "V0": [99.40, 98.39, 97.22, 92.79, 90.11, 86.60, 77.16, None],
        "V1": [99.50, 98.51, 97.53, 92.77, 90.48, 88.25, 77.88, 60.00],
    }
)


In [4]:
# Portfolio Definitions –––––

# Total Market Value in € mln
iTOTALMV = 1500.0

# Create DataFrame
dfPORTFOLIOS = pd.DataFrame(
    [
        {"Portfolio": "Investment Grade", "Rating": "AAA", "Weight": 0.60},
        {"Portfolio": "Investment Grade", "Rating": "AA",  "Weight": 0.30},
        {"Portfolio": "Investment Grade", "Rating": "BBB", "Weight": 0.10},
        {"Portfolio": "Junk", "Rating": "BB",  "Weight": 0.60},
        {"Portfolio": "Junk", "Rating": "B",   "Weight": 0.35},
        {"Portfolio": "Junk", "Rating": "CCC", "Weight": 0.05},
    ]
)


## **Concentrated portfolio**

In [5]:
# Settings –––––

# Correlations to run
vRHO = [0.00, 0.33, 0.66, 1.00]

# Monte Carlo draws
iN = 200_000

# Confidence levels for VaR/ES
vALPHA = [0.90, 0.995]

# Random seed for the main run
iSEED = 7
np.random.seed(iSEED)

# Rating order (best to worst)
sRATINGS_BEST = ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "D"]

# For threshold construction we want worst -> best from the LEFT tail (default is far left)
sRATINGS_WORST = ["D", "CCC", "B", "BB", "BBB", "A", "AA", "AAA"]


In [6]:
# Migration Threshold Construction –––––

# Build dictionary of thresholds
dTHRESHOLDS = {}

# Iterate over each starting rating
for sFROM in dfTRANSITIONS.index:

    # Take probabilities in worst to best order to build cumulative from the left tail
    vP = dfTRANSITIONS.loc[sFROM, sRATINGS_WORST].values.astype(float)

    # Cumulative probabilities from worst upward (left tail)
    vCUM = np.cumsum(vP)

    # Convert cumulative probabilities into z-thresholds
    dCUT = {}

    # Iterate over each destination rating (from worst to best)
    for j, sTO in enumerate(sRATINGS_WORST):

        # Cumulative probability for this rating
        fC = vCUM[j]

        # Numerical safety: clip to (tiny, 1-tiny) so norm.ppf doesn't explode accidentally
        # for AAA the cumulative should be 1.0, norm.ppf(1)=+inf
        if fC >= 1.0 - 1e-15: 
            fZ = np.inf
        # for D the cumulative should be 0.0, norm.ppf(0)=-inf
        elif fC <= 1e-15: 
            fZ = -np.inf
        # for all others compute normally
        else: 
            fZ = norm.ppf(fC)

        # Store cutoff
        dCUT[sTO] = fZ

    # Store in main dictionary
    dTHRESHOLDS[sFROM] = dCUT


In [7]:
# Model Validation –––––
# Required default threshold check for BBB -> D

# Analytic probability from the transition matrix
fP_BBB_D = float(dfTRANSITIONS.loc["BBB", "D"])
fZ_BBB_D_analytic = norm.ppf(fP_BBB_D)

# Default threshold from the constructed dictionary
fZ_BBB_D_code = dTHRESHOLDS["BBB"]["D"]

# Display results
print("Required default threshold check for BBB -> D")
print(f"P(BBB -> D) = {fP_BBB_D:.6f}")
print(f"Analytic z-threshold    = {fZ_BBB_D_analytic:.6f}")
print(f"Code z-threshold        = {fZ_BBB_D_code:.6f}")
print(f"Absolute difference     = {abs(fZ_BBB_D_analytic - fZ_BBB_D_code):.6f}")


Required default threshold check for BBB -> D
P(BBB -> D) = 0.001680
Analytic z-threshold    = -2.932726
Code z-threshold        = -2.932726
Absolute difference     = 0.000000


In [8]:
# Portfolio Preparation and Allocation –––––

# Ensure we can quickly lookup V0 and V1
dV0 = dict(zip(dfBONDVALUES["RATING"], dfBONDVALUES["V0"]))
dV1 = dict(zip(dfBONDVALUES["RATING"], dfBONDVALUES["V1"]))

# Build a concentrated issuer list per portfolio:
# Each row in dfISSUERS is one issuer (one per rating class in the portfolio)
vISSUERS = []

# Iterate over portfolios
for sPORT in dfPORTFOLIOS["Portfolio"].unique():

    # Filter portfolio ratings
    dfP = dfPORTFOLIOS[dfPORTFOLIOS["Portfolio"] == sPORT].copy()

    # Iterate over ratings in this portfolio
    for _, r in dfP.iterrows():
        # Fetch rating and weight
        sR0 = r["Rating"]
        fW = float(r["Weight"])

        # Compute allocated market value, bond price, and units held
        fMV0 = fW * float(iTOTALMV)
        fV0 = float(dV0[sR0])
        fUNITS = fMV0 / fV0

        # Store issuer info
        vISSUERS.append(
            {
                "Portfolio": sPORT,
                "R0": sR0,
                "Weight": fW,
                "MV0": fMV0,
                "V0": fV0,
                "UNITS": fUNITS,
            }
        )

# Convert to DataFrame
dfISSUERS = pd.DataFrame(vISSUERS)

# Display concentrated issuer setup
print("Concentrated issuer setup (one issuer per rating in portfolio):")
print(dfISSUERS)


Concentrated issuer setup (one issuer per rating in portfolio):
          Portfolio   R0  Weight    MV0     V0     UNITS
0  Investment Grade  AAA    0.60  900.0  99.40  9.054326
1  Investment Grade   AA    0.30  450.0  98.39  4.573636
2  Investment Grade  BBB    0.10  150.0  92.79  1.616554
3              Junk   BB    0.60  900.0  90.11  9.987793
4              Junk    B    0.35  525.0  86.60  6.062356
5              Junk  CCC    0.05   75.0  77.16  0.972006


In [9]:
# Export dfISSUERS to LaTeX –––––

with open("Documentation/Tables/ccsr-a1-issuers-concentrated.tex", "w") as f:
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\caption{Issuer Setup for Concentrated Portfolios}\n")
    f.write("\\label{tab:issuers_concentrated}\n")
    f.write("\\begin{tabular}{llrrrrr}\n")
    f.write("\\toprule\n")
    f.write("Portfolio & Rating & Weight & $\\text{MV}_{0}$ & $\\text{V}_{0}$ & Units \\\\\n")
    f.write("\\midrule\n")
    for _, row in dfISSUERS.iterrows():
        f.write(f"{row['Portfolio']} & {row['R0']} & {row['Weight']:.2f} & {row['MV0']:.1f} & {row['V0']:.2f} & {row['UNITS']:.6f} \\\\\n")
    f.write("\\bottomrule\n")
    f.write("\\end{tabular}\n")
    f.write("\\end{table}\n")

In [10]:
# Monte Carlo Simulation –––––
# For each portfolio and each correlation value

# Store results 
vRESULTS = []

# Iterate over portfolios
for sPORT in dfISSUERS["Portfolio"].unique():

    # Filter portfolio issuers
    dfP = dfISSUERS[dfISSUERS["Portfolio"] == sPORT].reset_index(drop=True)
    # Number of issuers in this portfolio (3 in this case)
    iK = dfP.shape[0]

    # Extract arrays of issuer params
    vR0 = dfP["R0"].tolist()
    vUNITS = dfP["UNITS"].values.astype(float)

    # For each rho value, run MC and compute metrics
    for fRHO in vRHO:

        # Draw common factor Y for all scenarios
        vY = np.random.normal(loc=0.0, scale=1.0, size=iN)

        # Draw idiosyncratic eps for each issuer (matrix iN x iK)
        mEPS = np.random.normal(loc=0.0, scale=1.0, size=(iN, iK))

        # Build X matrix
        # If rho=1, sqrt(1-rho)=0, so X_i = Y for everyone (perfect correlation)
        fA = np.sqrt(fRHO)
        fB = np.sqrt(1.0 - fRHO)

        # Matrix of X values (iN x iK)
        mX = fA * vY.reshape(-1, 1) + fB * mEPS  

        # Convert each issuer column into migrated rating, then into V1
        vV_PORT = np.zeros(iN, dtype=float)

        # Iterate over issuers
        for k in range(iK):

            # Fetch initial rating and X values for this issuer
            sR_INIT = vR0[k]
            vX = mX[:, k]

            # Fetch thresholds for this initial rating
            dCUT = dTHRESHOLDS[sR_INIT]

            # Now convert vX into V1 values using the cutoffs
            vV1_ISS = np.empty(iN, dtype=float)

            # Start with AAA by default (if above all cutoffs)
            vV1_ISS[:] = float(dV1["AAA"])

            # D
            mD = vX <= dCUT["D"]
            vV1_ISS[mD] = float(dV1["D"])

            # CCC (but not already D)
            mCCC = (vX > dCUT["D"]) & (vX <= dCUT["CCC"])
            vV1_ISS[mCCC] = float(dV1["CCC"])

            # B
            mB = (vX > dCUT["CCC"]) & (vX <= dCUT["B"])
            vV1_ISS[mB] = float(dV1["B"])

            # BB
            mBB = (vX > dCUT["B"]) & (vX <= dCUT["BB"])
            vV1_ISS[mBB] = float(dV1["BB"])

            # BBB
            mBBB = (vX > dCUT["BB"]) & (vX <= dCUT["BBB"])
            vV1_ISS[mBBB] = float(dV1["BBB"])

            # A
            mA = (vX > dCUT["BBB"]) & (vX <= dCUT["A"])
            vV1_ISS[mA] = float(dV1["A"])

            # AA
            mAA = (vX > dCUT["A"]) & (vX <= dCUT["AA"])
            vV1_ISS[mAA] = float(dV1["AA"])

            # Issuer value = UNITS * migrated V1
            vV_PORT += vUNITS[k] * vV1_ISS

        # Portfolio loss distribution in € mln
        vLOSS = float(iTOTALMV) - vV_PORT

        # Expected portfolio value (mean of V(t=1))
        fEV = float(np.mean(vV_PORT))

        # Compute VaR and ES at requested confidence levels
        # VaR at alpha = quantile(alpha) of loss
        # ES = mean(loss | loss >= VaR)
        dMET = {"EV": fEV}

        # Iterate over confidence levels
        for fA in vALPHA:

            # VaR = quantile of loss
            fVaR = float(np.quantile(vLOSS, fA))

            # Tail mask
            mTAIL = vLOSS >= fVaR
            fES = float(np.mean(vLOSS[mTAIL]))

            # Store with formatted string keys
            sA = f"{int(round(fA*1000))/10:.1f}%"
            dMET[f"VaR_{sA}"] = fVaR
            dMET[f"ES_{sA}"] = fES

        # Store results for this portfolio x rho
        vRESULTS.append(
            {
                "Portfolio": sPORT,
                "Rho": fRHO,
                "Expected Value": dMET["EV"],
                "90% VaR": dMET["VaR_90.0%"],
                "99.5% VaR": dMET["VaR_99.5%"],
                "90% ES": dMET["ES_90.0%"],
                "99.5% ES": dMET["ES_99.5%"],
            }
        )

# Final table for Question 1
dfQ1 = pd.DataFrame(vRESULTS)

# Make it look like the required output table order
dfQ1 = dfQ1.sort_values(["Portfolio", "Rho"]).reset_index(drop=True)

# Display results for Question 1
print("Concentrated — single issuer per rating")
print(dfQ1)


Concentrated — single issuer per rating
          Portfolio   Rho  Expected Value    90% VaR   99.5% VaR      90% ES  \
0  Investment Grade  0.00     1499.945915   5.884884   27.130707   11.704225   
1  Investment Grade  0.33     1499.975387   6.762133   33.794513   12.729194   
2  Investment Grade  0.66     1499.961105   5.884884   37.496420   14.447747   
3  Investment Grade  1.00     1499.982497  -1.421938   49.974574    0.488429   
4              Junk  0.00     1499.541189  48.468414  290.029707  109.116237   
5              Junk  0.33     1499.768007  48.468414  307.409178  112.317948   
6              Junk  0.66     1500.115567  48.468414  370.275806  122.890348   
7              Junk  1.00     1499.610142  48.468414  478.670725  147.202000   

     99.5% ES  
0   56.367254  
1   56.421773  
2   72.014173  
3   85.167830  
4  305.463794  
5  377.010625  
6  445.929820  
7  478.670725  


In [11]:
# Export dfQ1 to LaTeX –––––

with open("Documentation/Tables/ccsr-a1-results-concentrated.tex", "w") as f:
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\caption{Concentrated Portfolio Results}\n")
    f.write("\\label{tab:results_concentrated}\n")
    f.write("\\begin{tabular}{lrrrrrrr}\n")
    f.write("\\toprule\n")
    f.write("Portfolio & Rho ($\\rho$) & Expected Value & $\\text{VaR}_{0.90}$ & $\\text{VaR}_{0.995}$ & $\\text{ES}_{0.90}$ & $\\text{ES}_{0.995}$ \\\\\n")
    f.write("\\midrule\n")
    for _, row in dfQ1.iterrows():
        f.write(f"{row['Portfolio']} & {row['Rho']:.2f} & {row['Expected Value']:.4f} & {row['90% VaR']:.2f} & {row['99.5% VaR']:.2f} & {row['90% ES']:.2f} & {row['99.5% ES']:.2f} \\\\\n")
    f.write("\\bottomrule\n")
    f.write("\\end{tabular}\n")
    f.write("\\end{table}\n")

In [12]:
# Convergence Check –––––
# Required to verify convergence for Portfolio II (Junk) at rho = 33%, 3 times with different seeds

# Settings for convergence check
sPORT_CHECK = "Junk"
fRHO_CHECK = 0.33
vSEEDS = [1, 2, 3]

# Store results
vCHECK = []

# Extract issuer info once
dfPC = dfISSUERS[dfISSUERS["Portfolio"] == sPORT_CHECK].reset_index(drop=True)
iK = dfPC.shape[0]
vR0 = dfPC["R0"].tolist()
vUNITS = dfPC["UNITS"].values.astype(float)

# Iterate over seeds
for iS in vSEEDS:
    np.random.seed(iS)

    vY = np.random.normal(0.0, 1.0, iN)
    mEPS = np.random.normal(0.0, 1.0, (iN, iK))

    fA = np.sqrt(fRHO_CHECK)
    fB = np.sqrt(1.0 - fRHO_CHECK)
    mX = fA * vY.reshape(-1, 1) + fB * mEPS

    vV_PORT = np.zeros(iN, dtype=float)

    for k in range(iK):
        sR_INIT = vR0[k]
        vX = mX[:, k]
        dCUT = dTHRESHOLDS[sR_INIT]

        vV1_ISS = np.empty(iN, dtype=float)
        vV1_ISS[:] = float(dV1["AAA"])

        mD = vX <= dCUT["D"]
        vV1_ISS[mD] = float(dV1["D"])

        mCCC = (vX > dCUT["D"]) & (vX <= dCUT["CCC"])
        vV1_ISS[mCCC] = float(dV1["CCC"])

        mB = (vX > dCUT["CCC"]) & (vX <= dCUT["B"])
        vV1_ISS[mB] = float(dV1["B"])

        mBB = (vX > dCUT["B"]) & (vX <= dCUT["BB"])
        vV1_ISS[mBB] = float(dV1["BB"])

        mBBB = (vX > dCUT["BB"]) & (vX <= dCUT["BBB"])
        vV1_ISS[mBBB] = float(dV1["BBB"])

        mA = (vX > dCUT["BBB"]) & (vX <= dCUT["A"])
        vV1_ISS[mA] = float(dV1["A"])

        mAA = (vX > dCUT["A"]) & (vX <= dCUT["AA"])
        vV1_ISS[mAA] = float(dV1["AA"])

        vV_PORT += vUNITS[k] * vV1_ISS

    vLOSS = float(iTOTALMV) - vV_PORT
    fVaR_995 = float(np.quantile(vLOSS, 0.995))

    vCHECK.append({"Seed": iS, "N": iN, "Rho": fRHO_CHECK, "99.5% VaR": fVaR_995})

# Create DataFrame
dfCHECK_VaR = pd.DataFrame(vCHECK)

# Display convergence check results
print("Convergence Check for Portfolio II Junk, rho=33%, 3 seeds:")
print(dfCHECK_VaR)

# Calculate range of 99.5% VaR across seeds
fRANGE = float(dfCHECK_VaR["99.5% VaR"].max() - dfCHECK_VaR["99.5% VaR"].min())

# Display range
print(f"Observed 99.5% VaR range across seeds (N={iN:,}): {fRANGE:.6f} (in € mln)")


Convergence Check for Portfolio II Junk, rho=33%, 3 seeds:
   Seed       N   Rho   99.5% VaR
0     1  200000  0.33  300.088992
1     2  200000  0.33  307.409178
2     3  200000  0.33  300.088992
Observed 99.5% VaR range across seeds (N=200,000): 7.320186 (in € mln)


## **Diversified portfolio**

In [13]:
# Issuer-Level Portfolio Setup for Diversification –––––

# Number of issuers per rating class
iN_ISSUERS_PER_RATING = 100

vISSUERS = []

# Iterate over portfolios
for sPORT in dfPORTFOLIOS["Portfolio"].unique():
    dfP = dfPORTFOLIOS[dfPORTFOLIOS["Portfolio"] == sPORT].copy()

    # Iterate over ratings in this portfolio
    for _, r in dfP.iterrows():
        sR0 = r["Rating"]
        fW = float(r["Weight"])

        # Total market value allocated to this rating class
        fMV0_CLASS = fW * float(iTOTALMV)

        # Split equally among 100 issuers
        fMV0_PER_ISS = fMV0_CLASS / iN_ISSUERS_PER_RATING

        # Convert market value into "bond units" per issuer
        fV0 = float(dV0[sR0])
        fUNITS_PER_ISS = fMV0_PER_ISS / fV0

        # Create 100 issuers for this rating class
        for i in range(iN_ISSUERS_PER_RATING):
            vISSUERS.append(
                {
                    "Portfolio": sPORT,
                    "R0": sR0,
                    "Weight": fW,
                    "MV0_CLASS": fMV0_CLASS,
                    "MV0": fMV0_PER_ISS,          # per-issuer invested MV
                    "V0": fV0,
                    "UNITS": fUNITS_PER_ISS,      # per-issuer bond units
                    "IssuerID": f"{sPORT}_{sR0}_{i+1:03d}",
                }
            )

dfISSUERS_100 = pd.DataFrame(vISSUERS)


In [14]:
# Monte Carlo Simulation for Diversified Portfolios –––––
# Similar to before but now with 300 issuers per portfolio

# Store results
vRESULTS = []

# Iterate over portfolios
for sPORT in dfISSUERS_100["Portfolio"].unique():
    dfP = dfISSUERS_100[dfISSUERS_100["Portfolio"] == sPORT].reset_index(drop=True)

    iK = dfP.shape[0]
    vR0 = dfP["R0"].tolist()
    vUNITS = dfP["UNITS"].values.astype(float)

    # For each rho value, run MC and compute metrics
    for fRHO in vRHO:
        # Draw one systematic factor per scenario
        vY = np.random.normal(0.0, 1.0, iN)

        # Draw idiosyncratic shocks for each issuer (now many more columns)
        mEPS = np.random.normal(0.0, 1.0, (iN, iK))

        # Build asset returns matrix
        fA = np.sqrt(fRHO)
        fB = np.sqrt(1.0 - fRHO)
        mX = fA * vY.reshape(-1, 1) + fB * mEPS

        # Accumulate portfolio value scenario-by-scenario
        vV_PORT = np.zeros(iN, dtype=float)

        # Loop over issuers (same logic as Q1, just more issuers)
        for k in range(iK):
            sR_INIT = vR0[k]
            vX = mX[:, k]
            dCUT = dTHRESHOLDS[sR_INIT]

            # Assign migrated V1 per scenario using threshold intervals
            vV1_ISS = np.empty(iN, dtype=float)
            vV1_ISS[:] = float(dV1["AAA"])

            mD = vX <= dCUT["D"]
            vV1_ISS[mD] = float(dV1["D"])

            mCCC = (vX > dCUT["D"]) & (vX <= dCUT["CCC"])
            vV1_ISS[mCCC] = float(dV1["CCC"])

            mB = (vX > dCUT["CCC"]) & (vX <= dCUT["B"])
            vV1_ISS[mB] = float(dV1["B"])

            mBB = (vX > dCUT["B"]) & (vX <= dCUT["BB"])
            vV1_ISS[mBB] = float(dV1["BB"])

            mBBB = (vX > dCUT["BB"]) & (vX <= dCUT["BBB"])
            vV1_ISS[mBBB] = float(dV1["BBB"])

            mA = (vX > dCUT["BBB"]) & (vX <= dCUT["A"])
            vV1_ISS[mA] = float(dV1["A"])

            mAA = (vX > dCUT["A"]) & (vX <= dCUT["AA"])
            vV1_ISS[mAA] = float(dV1["AA"])

            # Issuer value contribution
            vV_PORT += vUNITS[k] * vV1_ISS

        # Loss distribution (positive loss = bad)
        vLOSS = float(iTOTALMV) - vV_PORT

        # Expected value
        fEV = float(np.mean(vV_PORT))

        # VaR / ES
        dOUT = {"Portfolio": sPORT, "Rho": fRHO, "Expected Value": fEV}

        # Iterate over confidence levels
        for fA in vALPHA:
            fVaR = float(np.quantile(vLOSS, fA))
            mTAIL = vLOSS >= fVaR
            fES = float(np.mean(vLOSS[mTAIL]))

            sA = f"{int(round(fA*1000))/10:.1f}%"
            dOUT[f"VaR_{sA}"] = fVaR
            dOUT[f"ES_{sA}"] = fES

        # Store results
        vRESULTS.append(
            {
                "Portfolio": dOUT["Portfolio"],
                "Rho": dOUT["Rho"],
                "Expected Value": dOUT["Expected Value"],
                "90% VaR": dOUT["VaR_90.0%"],
                "99.5% VaR": dOUT["VaR_99.5%"],
                "90% ES": dOUT["ES_90.0%"],
                "99.5% ES": dOUT["ES_99.5%"],
            }
        )

# Final table for Question 2
dfQ2 = pd.DataFrame(vRESULTS).sort_values(["Portfolio", "Rho"]).reset_index(drop=True)

# Display results for Question 2
print("Diversified — 100 issuers per rating:")
print(dfQ2)


Diversified — 100 issuers per rating:
          Portfolio   Rho  Expected Value    90% VaR   99.5% VaR      90% ES  \
0  Investment Grade  0.00     1499.961690   0.803920    2.194930    1.276378   
1  Investment Grade  0.33     1499.956604   3.217612   16.966627    7.346806   
2  Investment Grade  0.66     1499.956092   3.583234   31.346841   11.493582   
3  Investment Grade  1.00     1500.003706  -1.421938   49.974574    0.466691   
4              Junk  0.00     1499.927030   6.517068   14.295953    9.268482   
5              Junk  0.33     1499.919128  32.643769  137.514946   65.547506   
6              Junk  0.66     1499.920062  38.665552  265.980201  108.161632   
7              Junk  1.00     1500.098271  48.468414  478.670725  144.705701   

     99.5% ES  
0    2.588288  
1   24.555555  
2   52.719401  
3   84.100197  
4   16.315566  
5  172.636312  
6  325.429728  
7  478.670725  


In [15]:
# Export dfQ2 to LaTeX –––––

with open("Documentation/Tables/ccsr-a1-results-diversified.tex", "w") as f:
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\caption{Diversified Portfolio Results}\n")
    f.write("\\label{tab:results_diversified}\n")
    f.write("\\begin{tabular}{lrrrrrrr}\n")
    f.write("\\toprule\n")
    f.write("Portfolio & Rho ($\\rho$) & Expected Value & $\\text{VaR}_{0.90}$ & $\\text{VaR}_{0.995}$ & $\\text{ES}_{0.90}$ & $\\text{ES}_{0.995}$ \\\\\n")
    f.write("\\midrule\n")
    for _, row in dfQ2.iterrows():
        f.write(f"{row['Portfolio']} & {row['Rho']:.2f} & {row['Expected Value']:.4f} & {row['90% VaR']:.2f} & {row['99.5% VaR']:.2f} & {row['90% ES']:.2f} & {row['99.5% ES']:.2f} \\\\\n")
    f.write("\\bottomrule\n")
    f.write("\\end{tabular}\n")
    f.write("\\end{table}\n")

## **Analysis**

In [16]:
# Comparison of Concentrated vs Diversified Portfolio –––––

# Comparison table
dfCMP = (dfQ1.merge(dfQ2, on=["Portfolio", "Rho"], suffixes=("_Q1_conc", "_Q2_div")))

# Display comparison
print("Q1 vs Q2 comparison")
print(dfCMP[[
    "Portfolio","Rho",
    "99.5% VaR_Q1_conc","99.5% VaR_Q2_div",
    "99.5% ES_Q1_conc","99.5% ES_Q2_div"
]])


Q1 vs Q2 comparison
          Portfolio   Rho  99.5% VaR_Q1_conc  99.5% VaR_Q2_div  \
0  Investment Grade  0.00          27.130707          2.194930   
1  Investment Grade  0.33          33.794513         16.966627   
2  Investment Grade  0.66          37.496420         31.346841   
3  Investment Grade  1.00          49.974574         49.974574   
4              Junk  0.00         290.029707         14.295953   
5              Junk  0.33         307.409178        137.514946   
6              Junk  0.66         370.275806        265.980201   
7              Junk  1.00         478.670725        478.670725   

   99.5% ES_Q1_conc  99.5% ES_Q2_div  
0         56.367254         2.588288  
1         56.421773        24.555555  
2         72.014173        52.719401  
3         85.167830        84.100197  
4        305.463794        16.315566  
5        377.010625       172.636312  
6        445.929820       325.429728  
7        478.670725       478.670725  


In [17]:
# Export comparison to LaTeX –––––

with open("Documentation/Tables/ccsr-a1-results-comparison.tex", "w") as f:
    f.write("\\begin{table}[ht]\n")
    f.write("\\centering\n")
    f.write("\\caption{Comparison of Concentrated vs Diversified Portfolio Results}\n")
    f.write("\\label{tab:results_comparison}\n")
    f.write("\\begin{tabular}{l r r r r r}\n")
    f.write("\\toprule\n")
    # First header row (grouped)
    f.write(" & & \\multicolumn{2}{c}{$\\text{VaR}_{0.995}$} "
            "& \\multicolumn{2}{c}{$\\text{ES}_{0.995}$} \\\\\n")
    # Second header row (sub-columns)
    f.write("Portfolio & Rho ($\\rho$) & Concentrated & Diversified & Concentrated & Diversified \\\\\n")
    f.write("\\midrule\n")
    for _, row in dfCMP.iterrows():
        f.write(
            f"{row['Portfolio']} & {row['Rho']:.2f} & "
            f"{row['99.5% VaR_Q1_conc']:.2f} & {row['99.5% VaR_Q2_div']:.2f} & "
            f"{row['99.5% ES_Q1_conc']:.2f} & {row['99.5% ES_Q2_div']:.2f} \\\\\n"
        )
    f.write("\\bottomrule\n")
    f.write("\\end{tabular}\n")
    f.write("\\end{table}\n")


## **Code export**

In [18]:
# Export python code / markdown cells separately –––––

# Config –
NAME = "ccsr-a1"

# Import –
import nbformat

# Load notebook –
nb = nbformat.read(f"{NAME}-script.ipynb", as_version=4)

# Write code cells into one python script –
with open(f"Documentation/{NAME}-script.py", "w", encoding="utf-8") as f:
    for cell in nb.cells:
        if cell.cell_type == "code":
            f.write(cell.source.replace("–", "-") + "\n\n")


# Collect markdown cells –
md_cells = [cell['source'] for cell in nb.cells if cell['cell_type'] == 'markdown']

# Write them into one markdown file
with open(f"Documentation/{NAME}-markdown.md", "w") as f:
    f.write("\n\n".join(md_cells))


# Convert markdown to LaTeX using pandoc –
# rewrite and run in bash
# pandoc Documentation/{NAME}-markdown.md -o Documentation/{NAME}-markdown.tex